In [1]:
%pip install base58
%pip install cryptography

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePublicKey, EllipticCurve, SECP256K1, derive_private_key
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, hmac
from itertools import combinations
from pprint import pprint

import base58
import base64
import json

# 비트코인 지갑에서 비밀키 추출하기

문제에서 주어져 있는 (깨진) JSON 파일에서 공개키를 다음과 같이 얻는다.

In [3]:
xpubkey = base58.b58decode("xpub661MyMwAqRbcFwkbijMsskkrPEja9rZQAvGavNLGpthpwzbPyBDjCFUiLHVQXED2YM9pUAC7zz62ShWRPRdwbyyWEQ5CK1yP5vPWrmGCg7D")

[BIP-0032의 직렬화 형식](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki#Serialization_format)에 따라 직렬화된 공개키를 살펴보자.

In [4]:
print(f"version bytes (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[0:4]])}")
print(f"depth (1B)\t\t {' '.join([f'{x:02x}' for x in xpubkey[4:5]])}")
print(f"fingerprint (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[5:9]])}")
print(f"child number (4B)\t {' '.join([f'{x:02x}' for x in xpubkey[9:13]])}")
chain_code = xpubkey[13:45]
print(f"chain code (32B)\t {' '.join([f'{x:02x}' for x in chain_code])}")
public_key_data = xpubkey[45:78]
print(f"public key data (33B)\t {' '.join([f'{x:02x}' for x in public_key_data])}")

version bytes (4B)	 04 88 b2 1e
depth (1B)		 00
fingerprint (4B)	 00 00 00 00
child number (4B)	 00 00 00 00
chain code (32B)	 8c d2 ba bc c1 f6 71 bc 33 4e aa 09 13 a4 b3 10 d1 3e 40 8f a0 11 b8 63 6a 30 f1 2d 14 04 c6 b4
public key data (33B)	 03 fd 64 98 36 81 3f a9 b2 22 2e 6f 6a 4c 92 fa f6 42 e8 36 6e c6 5d 63 40 15 17 99 d8 43 69 95 33


Depth, fingerprint, child number 모두 0으로 채워져 있고, 주어진 정보는 master key의 것임을 알 수 있다.

Public key data를 SEC-1에 따라 파싱해보자.

In [5]:
curvepoint = EllipticCurvePublicKey.from_encoded_point(SECP256K1(), public_key_data)
curvepoint.public_numbers()

<EllipticCurvePublicNumbers(curve=secp256k1, x=114612885932937541895952981204780521972488172672556735824743693348595397399859, y=36275940037485766249167729003024982632875105444299785428776410948186178771039>

이 문제에서는 이 공개키에 대응하는 비밀키를 구하면 된다. BIP-0032의 ["Master key generation"](https://github.com/bitcoin/bips/blob/master/bip-0032.mediawiki#master-key-generation)에 따르면 마스터키는 다음과 같은 과정으로 생성된다.

1. 어떤 길이의 Seed `S`를 생성한다. (이 문제에서는 여기서 BIP-0039를 따랐다.)
2. 64byte sequece `I = HMAC_SHA512(key="Bitcoin seed", data=S)`를 계산한다.
3. `I`의 첫 32byte를 `I_L`, 다음 32byte를 `I_R`이라고 한다.
4. `I_L`은 비밀키, `I_R`은 chain code로 사용된다.

방금 chain code를 `xpubkey`에서 구했으므로, `I_R`을 구한 것과 마찬가지이다.

In [6]:
I_R = chain_code

지갑 정보에 있는 `xPrivKeyEncrypted`와 `mnemonicEncrypted`는 무엇일까? 여기 저장된 정보를 파싱해보자.

In [7]:
xprivkey_encrypted = json.loads("""{
\"IV\":\"TGOpwxj3UiffLawxlO8P0Q==\",
\"V\":1,
\"Key Derivation Iteration\":1000,
\"Key Length\":128,
\"Tag Size\":64,
\"Adata\":\"\",
\"Cipher\":\"AES-CCM\",
\"Salt\":\"2B2CnAzrhrU=\",
\"Cipher Text\":\"kN197TSnBiyqHv+Ul1ioNdvmNZV3zDSkane+qTrLKLoJaeTh2mUooYKYY+EgztWp6ichJfqUWCM0D9Yd72j4Ytj4wVLVRP+5VcUBqpnHli2gVIYIETocig92bNCzIZdb42jheXbRd+EvH5ZSanq3Sr3uQJN/eN0=\"}""")
pprint(xprivkey_encrypted)
mnemonic_encrypted = json.loads("""{\"IV\":\"2k+eN8VqCnilue22ENpdfQ==\",
\"V\":1,
\"Key Derivation Iteration\":1000,
\"Key Length\":128,
\"Tag Size\":64,
\"Adata\":\"\",
\"Cipher\":\"AES-CCM\",
\"Salt\":\"2B2CnAzrhrU=\",
\"ct\":\"NjuugzjFTbX7Tj05w4FVpPnyP9lru7uFtPRPwkn1nQGprvFirzHSjLVCipWEJqUayFb/Ksm46yIWtbPCTF0viJUD4+lcBcSlpMpBuwxBc92yUaQ5aE8lX21s\"}""")
pprint(mnemonic_encrypted)

{'Adata': '',
 'Cipher': 'AES-CCM',
 'Cipher Text': 'kN197TSnBiyqHv+Ul1ioNdvmNZV3zDSkane+qTrLKLoJaeTh2mUooYKYY+EgztWp6ichJfqUWCM0D9Yd72j4Ytj4wVLVRP+5VcUBqpnHli2gVIYIETocig92bNCzIZdb42jheXbRd+EvH5ZSanq3Sr3uQJN/eN0=',
 'IV': 'TGOpwxj3UiffLawxlO8P0Q==',
 'Key Derivation Iteration': 1000,
 'Key Length': 128,
 'Salt': '2B2CnAzrhrU=',
 'Tag Size': 64,
 'V': 1}
{'Adata': '',
 'Cipher': 'AES-CCM',
 'IV': '2k+eN8VqCnilue22ENpdfQ==',
 'Key Derivation Iteration': 1000,
 'Key Length': 128,
 'Salt': '2B2CnAzrhrU=',
 'Tag Size': 64,
 'V': 1,
 'ct': 'NjuugzjFTbX7Tj05w4FVpPnyP9lru7uFtPRPwkn1nQGprvFirzHSjLVCipWEJqUayFb/Ksm46yIWtbPCTF0viJUD4+lcBcSlpMpBuwxBc92yUaQ5aE8lX21s'}


둘다 AES-CCM으로 암호화된 정보이고, 흥미롭게도 `Salt`값이 같다.

In [8]:
xprivkey_encrypted["Salt"] == mnemonic_encrypted["Salt"]

True

AES-CCM 암호화에서 암호문의 길이는 (plaintext length)+(tag length)이므로, mnemonic sentence의 길이를 구할 수 있다.

In [9]:
len(base64.b64decode(mnemonic_encrypted["ct"])) - mnemonic_encrypted["Tag Size"] // 8

82

문제에서 주어진 dictionary는 15개의 단어만 포함되어 있으므로, mnemonic sentence의 길이가 82가 되게 하는 단어 조합의 개수는 얼마 되지 않는다. 직접 구해보자.

In [10]:
mnemonic_words = [
    (0, "abandon"),
    (224, "bright"),
    (248, "business"),
    (365, "color"),
    (958, "jelly"),
    (964, "joy"),
    (1033, "license"),
    (1114, "mercy"),
    (1156, "mountain"),
    (1293, "payment"),
    (1354, "power"),
    (1358, "prefer"),
    (1401, "quality"),
    (1798, "this"),
    (2047, "zoo"),
]

dictionary의 모든 단어를 이어 만든 문장의 길이는 다음과 같다.

In [11]:
len(" ".join(map(lambda x: x[1], mnemonic_words)))

100

BIP-0032에 의하면 dictionary에서 12개의 단어를 고른다. 따라서 `mnemonic_words`에서 길이 합이 100 - 82 - 3 = 15인 단어 3개의 조합을 찾으면 된다.

In [12]:
unused_triples = list(filter(lambda t: sum([len(w[1]) for w in t]) == 15, combinations(mnemonic_words, 3)))
len(unused_triples)

50

사용되지 않는 조합을 제외하면 단어의 조합 개수는 50개로 줄어든다.

In [13]:
used_words = [set(mnemonic_words) - set(t) for t in unused_triples]

각 조합에 12개의 단어가 있으므로, 50\*12!개 경우를 탐색하면 된다. 탐색으로 얻은 결과를 대입해 보자.

In [14]:
sentence = b"license business color this prefer joy payment jelly mountain quality power bright"
kdf = PBKDF2HMAC(algorithm=hashes.SHA512(), length=64, salt=b"mnemonic", iterations=2048)
seed = kdf.derive(sentence)
h = hmac.HMAC(b"Bitcoin seed", hashes.SHA512())
h.update(seed)
I = h.finalize()
I[32:] == I_R

True

비밀키를 구해보자.

In [15]:
privkey_bytes = I[:32]
privkey = 0
for byte in privkey_bytes:
    privkey *= 2 ** 8
    privkey += byte
privkey

106803545498616358317922859232323094583340750393811153506629572524687868755802

과연 이 키가 우리가 찾는 것이었을까?

In [16]:
derive_private_key(privkey, SECP256K1()).public_key().public_numbers() == curvepoint.public_numbers()

True

주어진 공개키에 대응되는 비밀키임을 알 수 있다. PROFIT!